In [1]:
from dask_jobqueue import SLURMCluster 
from dask.distributed import Client 
  
cluster = SLURMCluster(cores=28,name='make_profiles',walltime='00:30:00',job_extra=['--constraint=HSW24','--exclusive','--nodes=1'],memory='120GB',interface='ib0') 
cluster.scale(168)
cluster

/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/bokeh/core.py:74: UserWarning: 
Failed to start diagnostics server on port 8787. [Errno 13] Permission denied
  warnings.warn("\n" + msg)


In [2]:
from dask.distributed import Client
client = Client(cluster)
client

Client Scheduler: tcp://172.30.100.4:44907 Dashboard: http://172.30.100.4:36034/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [3]:
!squeue -u albert7a

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           8251700     hsw24 make_pro albert7a  R       0:00      1 n2067
           8251701     hsw24 make_pro albert7a  R       0:00      1 n2068
           8251702     hsw24 make_pro albert7a  R       0:00      1 n2069
           8251703     hsw24 make_pro albert7a  R       0:00      1 n2070
           8251704     hsw24 make_pro albert7a  R       0:00      1 n2071
           8251705     hsw24 make_pro albert7a  R       0:00      1 n2072
           8251698     hsw24 make_pro albert7a  R       0:14      1 n2065
           8251699     hsw24 make_pro albert7a  R       0:14      1 n2066
           8251695     hsw24 make_pro albert7a  R       0:17      1 n2062
           8251696     hsw24 make_pro albert7a  R       0:17      1 n2063
           8251697     hsw24 make_pro albert7a  R       0:17      1 n2064


In [4]:
import time
nb_workers = 0
while True:
    nb_workers = len(client.scheduler_info()["workers"])
    if nb_workers >= 2:
        break
    time.sleep(1)
print(nb_workers)

166


In [5]:
!squeue -u albert7a

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           8251700     hsw24 make_pro albert7a  R       0:11      1 n2067
           8251701     hsw24 make_pro albert7a  R       0:11      1 n2068
           8251702     hsw24 make_pro albert7a  R       0:11      1 n2069
           8251703     hsw24 make_pro albert7a  R       0:11      1 n2070
           8251704     hsw24 make_pro albert7a  R       0:11      1 n2071
           8251705     hsw24 make_pro albert7a  R       0:11      1 n2072
           8251698     hsw24 make_pro albert7a  R       0:25      1 n2065
           8251699     hsw24 make_pro albert7a  R       0:25      1 n2066
           8251695     hsw24 make_pro albert7a  R       0:28      1 n2062
           8251696     hsw24 make_pro albert7a  R       0:28      1 n2063
           8251697     hsw24 make_pro albert7a  R       0:28      1 n2064


In [6]:
import os
import sys
import numpy as np
import numpy.ma as ma
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.feature as cfeature
import cmocean
import glob

%matplotlib inline


/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/xarray/core/merge.py:16: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  PANDAS_TYPES = (pd.Series, pd.DataFrame, pd.Panel)


In [7]:
#params

plot_eke_monthly=False
plot_eke_annual=True

region='EU'

In [8]:
if region == 'EU':
    imin=3858
    imax=5076
    jmin=424
    jmax=3286
    
#cdffindij -w -80.418  -40.000   26.991   47.986 -c /store/molines/NATL60/NATL60-I/NATL60_coordinates_v4.nc
if region == 'GS':
    imin=63
    imax=2507
    jmin=29
    jmax=1611

In [9]:
config='NATL60'
case='CJM165'
freq='1d'
#year='2010' #depends on the month

datadir='/store/molines/'+str(config)+'/'+str(config)+'-'+str(case)+'-S/'+str(freq)+'/'
#!ls $datadir
gridfile='/store/molines/NATL60/NATL60-I/NATL60_coordinates_v4.nc'
maskfile='/store/molines/NATL60/NATL60-I/NATL60_v4.1_cdf_byte_mask.nc'

In [10]:
dsgrid=xr.open_dataset(gridfile)
dsmask=xr.open_dataset(maskfile)

In [11]:
def compute_TKE(u2mean,v2mean):
    ua=0.5*(u2mean+u2mean.shift(x=1))
    va=0.5*(v2mean+v2mean.shift(y=1))
    uat=ua.rename({'depthu':'deptht'})
    vat=va.rename({'depthv':'deptht'})
    tke=0.5*(uat+vat)
    return tke


In [12]:
month_name={1:'January',2:'February',3:'March',4:'April',5:'May',6:'June',7:'July',8:'August',9:'September',10:'October',11:'November',12:'December'}
def plot_one_month(var,mask,vmin,vmax,cmap,title,figfile,month,year):
    if region == 'EU':
        fig, ax = plt.subplots(1,1,figsize=(10,15))
    if region == 'GS':
        fig, ax = plt.subplots(1,1,figsize=(15,10))
    var.where(mask==1).plot(cmap=cmap,ax=ax,vmin=vmin,vmax=vmax,add_labels=True)
    ax.set_title(config+'-'+case+' '+month_name[month])
    fig.suptitle('Monthly mean of '+str(title))
    plt.savefig('Plots/'+config+'/Maps/'+region+'/'+config+'-'+case+'y'+str(year)+'m'+str(month)+'_'+figfile+'.png')

In [13]:
def plot_year(var,mask,vmin,vmax,cmap,title,figfile,year):
    if region == 'EU':
        fig, ax = plt.subplots(1,1,figsize=(10,15))
    if region == 'GS':
        fig, ax = plt.subplots(1,1,figsize=(15,10))
    var.where(mask==1).plot(cmap=cmap,ax=ax,vmin=vmin,vmax=vmax,add_labels=True)
    ax.set_title(config+'-'+case+' '+year)
    fig.suptitle('Monthly mean of '+str(title))
    plt.savefig('Plots/'+config+'/Maps/'+region+'/'+config+'-'+case+'y'+str(year)+'_'+figfile+'.png')

In [14]:
def plot_all_month(year):
    filesU=sorted(glob.glob(datadir+'/*/'+str(config)+'-'+str(case)+'_y????m??d??.'+str(freq)+'_gridU.nc'))
    filesV=sorted(glob.glob(datadir+'/*/'+str(config)+'-'+str(case)+'_y????m??d??.'+str(freq)+'_gridV.nc'))
    dsU=xr.open_mfdataset(filesU, chunks={'x':700,'y':1000,'time_counter':10,'depthu':1})
    dsV=xr.open_mfdataset(filesV, chunks={'x':700,'y':1000,'time_counter':10,'depthv':1})
    if year == '2013':
        u=dsU.sel(time_counter=slice('2013-01-01','2013-09-30')).vozocrtx[:,:,jmin:jmax,imin:imax]
        v=dsV.sel(time_counter=slice('2013-01-01','2013-09-30')).vomecrty[:,:,jmin:jmax,imin:imax]
    if year == '2012':
        u=dsU.sel(time_counter=slice('2012-07-01','2012-12-31')).vozocrtx[:,:,jmin:jmax,imin:imax]
        v=dsV.sel(time_counter=slice('2012-07-01','2012-12-31')).vomecrty[:,:,jmin:jmax,imin:imax]
    mask=dsmask.tmask[0,:,jmin:jmax,imin:imax]
    u2=u*u
    v2=v*v
    u2mean=u2.mean(dim='time_counter')
    v2mean=v2.mean(dim='time_counter')
    umonth=u.groupby('time_counter.month').mean('time_counter')
    vmonth=v.groupby('time_counter.month').mean('time_counter')
    umonth2=umonth*umonth
    vmonth2=vmonth*vmonth
    tke=compute_TKE(umonth2,vmonth2)
    for m in np.arange(1,10):
        if not os.path.exists('Plots/'+config+'/Maps/'+region+'/'+config+'-'+case+'y'+str(year)+'m'+str(m)+'_tke_surf.png'):
            print('Plotting for month '+str(m))
            plot_one_month(10000*tke[m-1,0],mask[0],0,2500,'cmo.amp','Surface TKE y'+year,'tke_surf',m,year)
 

In [15]:
def process_plot_year():
    filesU=sorted(glob.glob(datadir+'/*/'+str(config)+'-'+str(case)+'_y????m??d??.'+str(freq)+'_gridU.nc'))
    filesV=sorted(glob.glob(datadir+'/*/'+str(config)+'-'+str(case)+'_y????m??d??.'+str(freq)+'_gridV.nc'))
    dsU=xr.open_mfdataset(filesU, chunks={'x':700,'y':1000,'time_counter':10,'depthu':1})
    dsV=xr.open_mfdataset(filesV, chunks={'x':700,'y':1000,'time_counter':10,'depthv':1})
    u=dsU.sel(time_counter=slice('2012-10-01','2013-09-30')).vozocrtx[:,:,jmin:jmax,imin:imax]
    v=dsV.sel(time_counter=slice('2012-10-01','2013-09-30')).vomecrty[:,:,jmin:jmax,imin:imax]
    mask=dsmask.tmask[0,:,jmin:jmax,imin:imax]
    u2=u*u
    v2=v*v
    u2mean=u2.mean(dim='time_counter')
    v2mean=v2.mean(dim='time_counter')
    umonth=u.groupby('time_counter.month').mean('time_counter')
    vmonth=v.groupby('time_counter.month').mean('time_counter')
    umonth2=umonth*umonth
    vmonth2=vmonth*vmonth
    tke=compute_TKE(umonth2,vmonth2)
    if not os.path.exists('Plots/'+config+'/Maps/'+region+'/'+config+'-'+case+'y2012-2013_tke_surf.png'):
        print('Plotting for year 2012-2013')
        plot_year(10000*tke.mean(dim='month')[0],mask[0],0,2500,'cmo.amp','Surface TKE y2012-2013','tke_surf','2012-2013')
 

In [16]:
if plot_eke_monthly == True:
    for year in ['2013','2012']:
        plot_all_month(year)


In [ ]:
if plot_eke_annual == True:
    process_plot_year()
